## Imports

In [1]:
import pandas as pd
import numpy as np
import re

## Import Data

In [20]:
titles_imdb = pd.read_csv('../data/title.basics.tsv.gz', sep='\t', compression='gzip')
ratings_imdb = pd.read_csv('../data/title.ratings.tsv.gz', sep='\t', compression='gzip')
budgets_kaggle = pd.read_csv('../data/kaggle_movies_metadata.csv')

C:\Users\gabri\AppData\Local\Temp\ipykernel_10728\1546009065.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  titles_imdb = pd.read_csv('../data/title.basics.tsv.gz', sep='\t', compression='gzip')
C:\Users\gabri\AppData\Local\Temp\ipykernel_10728\1546009065.py:3: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  budgets_kaggle = pd.read_csv('../data/kaggle_movies_metadata.csv')


In [21]:
titles_imdb.head(3)

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"


In [22]:
ratings_imdb.head(3)

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1736


In [23]:
budgets_kaggle.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [24]:
# Importing CMU Datasets
movie = pd.read_csv('./../data/MovieSummaries/movie.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_name',
                        'Movie_release_date',
                        'Movie_box_office_revenue',
                        'Movie_runtime',
                        'Movie_languages',
                        'Movie_countries',
                        'Movie_genres'],
                 header=None)

character = pd.read_csv('./../data/MovieSummaries/character.metadata.tsv', 
                 sep='\t',
                 names=['Wikipedia_movie_ID',
                        'Freebase_movie_ID',
                        'Movie_release_date',
                        'Character_name',
                        'Actor_date_of_birth',
                        'Actor_gender',
                        'Actor_height',
                        'Actor_ethnicity',
                        'Actor_name',
                        'Actor_age_at_movie_release',
                        'Freebase_character/actor_map_ID',
                        'Freebase_character_ID',
                        'Freebase_actor_ID'],
                 header=None)

In [25]:
movie.head(3)

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_languages,Movie_countries,Movie_genres
0,975900,/m/03vyhn,Ghosts of Mars,2001-08-24,14010832.0,98.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/01jfsb"": ""Thriller"", ""/m/06n90"": ""Science..."
1,3196793,/m/08yl5d,Getting Away with Murder: The JonBenét Ramsey ...,2000-02-16,NaN,95.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}","{""/m/02n4kr"": ""Mystery"", ""/m/03bxz7"": ""Biograp..."
2,28463795,/m/0crgdbh,Brun bitter,1988,NaN,83.0,"{""/m/05f_3"": ""Norwegian Language""}","{""/m/05b4w"": ""Norway""}","{""/m/0lsxr"": ""Crime Fiction"", ""/m/07s9rl0"": ""D..."


In [26]:
character.head(3)

,Wikipedia_movie_ID,Freebase_movie_ID,Movie_release_date,Character_name,Actor_date_of_birth,Actor_gender,Actor_height,Actor_ethnicity,Actor_name,Actor_age_at_movie_release,Freebase_character/actor_map_ID,Freebase_character_ID,Freebase_actor_ID
0,975900,/m/03vyhn,2001-08-24,Akooshay,1958-08-26,F,1.620,NaN,Wanda De Jesus,42.0,/m/0bgchxw,/m/0bgcj3x,/m/03wcfv7
1,975900,/m/03vyhn,2001-08-24,Lieutenant Melanie Ballard,1974-08-15,F,1.780,/m/044038p,Natasha Henstridge,27.0,/m/0jys3m,/m/0bgchn4,/m/0346l4
2,975900,/m/03vyhn,2001-08-24,Desolation Williams,1969-06-15,M,1.727,/m/0x67,Ice Cube,32.0,/m/0jys3g,/m/0bgchn_,/m/01vw26l


# Data cleaning

### Date change

In [27]:
#move release dates and actor birth dates to datetime format and keep only release year
character.Movie_release_date = pd.to_datetime(character['Movie_release_date'],
                                              errors='coerce').dt.year

movie.Movie_release_date = pd.to_datetime(movie['Movie_release_date'],
                                              errors='coerce').dt.year

In [28]:
#sort dataframes by ascending release year
character.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

movie.sort_values(by=['Movie_release_date'], ascending=True, inplace=True)

### Null value removal

In [29]:
# NaN percentage for character dataset
character.isnull().sum() * 100 / character.shape[0]

Wikipedia_movie_ID                  0.000000
Freebase_movie_ID                   0.000000
Movie_release_date                  2.221142
Character_name                     57.220488
Actor_date_of_birth                23.552763
Actor_gender                       10.120288
Actor_height                       65.645740
Actor_ethnicity                    76.466542
Actor_name                          0.272484
Actor_age_at_movie_release         35.084064
Freebase_character/actor_map_ID     0.000000
Freebase_character_ID              57.218269
Freebase_actor_ID                   0.180842
dtype: float64

In [31]:
character = character[['Wikipedia_movie_ID', 'Actor_name', 'Actor_gender', 'Actor_date_of_birth']]
character.head(3)

,Wikipedia_movie_ID,Actor_name,Actor_gender,Actor_date_of_birth
123565,1082508,Harriet Hartley,F,NaN
123564,1082508,Joseph Whitley,M,NaN
123563,1082508,Sarah Whitley,F,NaN


In [32]:
char_no_nan = character.dropna(axis=0, how='any')
char_no_nan

,Wikipedia_movie_ID,Actor_name,Actor_gender,Actor_date_of_birth
72684,644824,William Dickson,M,1860-08-03
294940,73372,John Ott,M,1909-10-23
342275,786716,William Dickson,M,1860-08-03
6955,10104497,James J. Corbett,M,1866-09-01
229796,1090697,Fred Ott,M,1860
...,...,...,...,...
450333,29119097,Deepankar De,M,1944-07-05
450494,17198681,Melissa Errico,F,1970-03-23
450495,17198681,Charlie Sheen,M,1965-09-03
450496,17198681,Renée Estevez,F,1967-04-02


In [33]:
# Nan percentage for movie dataset
movie.isnull().sum() * 100 / movie.shape[0]

Wikipedia_movie_ID           0.000000
Freebase_movie_ID            0.000000
Movie_name                   0.000000
Movie_release_date           8.444966
Movie_box_office_revenue    89.722416
Movie_runtime               25.018045
Movie_languages              0.000000
Movie_countries              0.000000
Movie_genres                 0.000000
dtype: float64

In [42]:
movie = movie[['Wikipedia_movie_ID', 'Movie_name', 'Movie_release_date', 'Movie_box_office_revenue','Movie_runtime', 'Movie_countries', 'Movie_genres']]

In [41]:
movie_no_nan = movie.dropna(0, inplace= False, how='any')

C:\Users\gabri\AppData\Local\Temp\ipykernel_10728\561346970.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  movie_no_nan = movie.dropna(0, inplace= False, how='any')


In [39]:
budgets_kaggle = budgets_kaggle[['original_title', 'budget']]

In [38]:
budgets_kaggle['budget'] = budgets_kaggle['budget'].replace('0', np.nan)
budgets_kaggle.isna().sum()

original_title        0
budget            36573
dtype: int64

In [40]:
budget_no_nan = budgets_kaggle.dropna()

# Merging Datasets

In [43]:
# Merging movie titles dataset with ratings dataset
first_merge = titles_imdb.merge(ratings_imdb, how='inner')
first_merge.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres,averageRating,numVotes
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short",5.7,1922
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short",5.8,259
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance",6.5,1736
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short",5.6,174
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short",6.2,2549


In [44]:
# Droping unnecessary columns
imdb = first_merge[['primaryTitle', 'averageRating', 'numVotes']]
imdb.head()

,primaryTitle,averageRating,numVotes
0,Carmencita,5.7,1922
1,Le clown et ses chiens,5.8,259
2,Pauvre Pierrot,6.5,1736
3,Un bon bock,5.6,174
4,Blacksmith Scene,6.2,2549


In [49]:
# Removing duplicate film titles and averaging the duplicate ratings
mean_rating_imdb = first_merge[['primaryTitle', 'averageRating']].groupby('primaryTitle').mean()
mean_rating_imdb.head(3)

,averageRating
primaryTitle,
!Next?,5.2
!Que ve el Bisbe!,6.2
!Women Art Revolution,6.8


In [50]:
# Removing duplicate film titles and adding together the duplicate votes
total_vote_imdb = first_merge[['primaryTitle', 'numVotes']].groupby('primaryTitle').sum()
total_vote_imdb.head(3)

,numVotes
primaryTitle,
!Next?,18
!Que ve el Bisbe!,12
!Women Art Revolution,247


In [51]:
# Merging the two above dataframes
no_dupe_imdb = mean_rating_imdb.merge(total_vote_imdb, 'inner', 'primaryTitle')
no_dupe_imdb.head(3)

,averageRating,numVotes
primaryTitle,,
!Next?,5.2,18
!Que ve el Bisbe!,6.2,12
!Women Art Revolution,6.8,247


In [52]:
# Merging the rating/votes dataframe with movie metadata with no nan values
rated_data = movie_no_nan.merge(no_dupe_imdb, 'inner', left_on='Movie_name', right_on='primaryTitle')
rated_data.head(3)

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,Movie_countries,Movie_genres,averageRating,numVotes
0,1762869,The Squaw Man,1914.0,244700.0,74.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0hfjk"": ""Weste...",5.833333,1418
1,3333,The Birth of a Nation,1915.0,50000000.0,190.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",6.600000,46215
2,4499436,Way Down East,1920.0,4500000.0,147.0,"{""/m/09c7w0"": ""United States of America""}","{""/m/06ppq"": ""Silent film"", ""/m/0219x_"": ""Indi...",6.750000,5798


## Parsing the genres and movie countries

In [53]:
rated_data['genres'] = rated_data['Movie_genres'].apply(lambda x : re.split(r':|,', x)[1::2])
rated_data['countries'] = rated_data['Movie_countries'].apply(lambda x : re.split(r':|,', x)[1::2])
rated_dat = rated_data.drop(['Movie_countries', 'Movie_genres'], axis = 1) #run once

In [54]:
rated_dat.head(3)

,Wikipedia_movie_ID,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries
0,1762869,The Squaw Man,1914.0,244700.0,74.0,5.833333,1418,"[ ""Silent film"", ""Western"", ""Drama"", ""Indie...","[ ""United States of America""}]"
1,3333,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46215,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
2,4499436,Way Down East,1920.0,4500000.0,147.0,6.750000,5798,"[ ""Silent film"", ""Indie"", ""Black-and-white"",...","[ ""United States of America""}]"


In [56]:
data = pd.merge(char_no_nan, rated_dat)
data = data.drop(['Wikipedia_movie_ID'], axis=1)
data.head(3)

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries
0,Dustin Farnum,M,1874-05-27,The Squaw Man,1914.0,244700.0,74.0,5.833333,1418,"[ ""Silent film"", ""Western"", ""Drama"", ""Indie...","[ ""United States of America""}]"
1,Elmer Clifton,M,1890-03-14,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46215,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"
2,Robert Harron,M,1893-04-12,The Birth of a Nation,1915.0,50000000.0,190.0,6.600000,46215,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]"


In [57]:
# Renaming column to avoid merge issues
budget_no_nan = budget_no_nan.rename(columns={'original_title': 'Movie_name'})
budget_no_nan.head(3)

,Movie_name,budget
0,Toy Story,30000000
1,Jumanji,65000000
3,Waiting to Exhale,16000000


In [58]:
# Merging dataset with budget information and the ratings dataset
budget = pd.merge(budget_no_nan, rated_dat)
budget.head(3)

,Movie_name,budget,Wikipedia_movie_ID,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries
0,Toy Story,30000000,53085,1995.0,361958736.0,77.0,7.781818,991078,"[ ""Buddy film"", ""Adventure"", ""Children's/Fam...","[ ""United States of America""}]"
1,Jumanji,65000000,3700174,1995.0,262797249.0,104.0,6.360000,348294,"[ ""Thriller"", ""Fantasy Adventure"", ""Japanese...","[ ""United States of America"", ""Japan""}]"
2,Waiting to Exhale,16000000,972970,1995.0,81452156.0,121.0,7.600000,16244,"[ ""Ensemble Film"", ""Female buddy film"", ""Com...","[ ""United States of America""}]"


In [63]:
# Converting the budget column from string to numerical value
budget['budget'] = pd.to_numeric(budget['budget'])

In [64]:
# Creating a Profit column
budget['profit'] = budget['Movie_box_office_revenue'] - budget['budget']
budget.head(3)

,Movie_name,budget,Wikipedia_movie_ID,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,profit
0,Toy Story,30000000,53085,1995.0,361958736.0,77.0,7.781818,991078,"[ ""Buddy film"", ""Adventure"", ""Children's/Fam...","[ ""United States of America""}]",331958736.0
1,Jumanji,65000000,3700174,1995.0,262797249.0,104.0,6.360000,348294,"[ ""Thriller"", ""Fantasy Adventure"", ""Japanese...","[ ""United States of America"", ""Japan""}]",197797249.0
2,Waiting to Exhale,16000000,972970,1995.0,81452156.0,121.0,7.600000,16244,"[ ""Ensemble Film"", ""Female buddy film"", ""Com...","[ ""United States of America""}]",65452156.0


In [65]:
# Merging the profit dataset with character dataset
profit = pd.merge(char_no_nan, budget)
profit = profit.drop(['Wikipedia_movie_ID'], axis=1)
profit.head(3)

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,budget,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,profit
0,Elmer Clifton,M,1890-03-14,The Birth of a Nation,100000,1915.0,50000000.0,190.0,6.6,46215,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",49900000.0
1,Elmer Clifton,M,1890-03-14,The Birth of a Nation,8500000,1915.0,50000000.0,190.0,6.6,46215,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",41500000.0
2,Robert Harron,M,1893-04-12,The Birth of a Nation,100000,1915.0,50000000.0,190.0,6.6,46215,"[ ""Silent film"", ""Indie"", ""Costume drama"", ...","[ ""United States of America""}]",49900000.0


### Adjusting for inflation

In [66]:
cf = pd.read_csv('../data/inflation_correction.csv') #inflation correction

inflation_match_profit = profit.merge(cf, how = 'inner', left_on='Movie_release_date', right_on='Year')
profit['profit_adj'] = profit['profit']/inflation_match_profit['CF']
inflation_match_data = data.merge(cf, how = 'inner', left_on='Movie_release_date', right_on='Year')
data['box_office_adj'] = data['Movie_box_office_revenue']/inflation_match_data['CF']

display(profit.sample(), data.sample())

,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,budget,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,profit,profit_adj
31936,Kerry Washington,F,1977-01-31,Ray,40000000,2004.0,124731534.0,178.0,7.42,157797,"[ ""Musical"", ""Biography"", ""Drama"", ""Musical...","[ ""United States of America""}]",84731534.0,1.072551e+08


,Actor_name,Actor_gender,Actor_date_of_birth,Movie_name,Movie_release_date,Movie_box_office_revenue,Movie_runtime,averageRating,numVotes,genres,countries,box_office_adj
77750,Chloë Moretz,F,1997-02-10,Dark Shadows,2012.0,238727149.0,113.0,6.755556,277974,"[ ""Thriller"", ""Comedy-drama"", ""Horror"", ""Dr...","[ ""United States of America""}]",2.486741e+08


In [67]:
profit.to_csv('../data/profit_data.csv') #useful for movie regression
data.to_csv('../data/data.csv')